In [ ]:
#r "nuget: Shioaji, 1.1.8"

In [2]:
using System.IO;
using Sinopac.Shioaji;
using System.Text.Json;
Shioaji _api = new Shioaji();

string jsonString = File.ReadAllText(@"D:\Sinopac.json");
JsonElement root = JsonDocument.Parse(jsonString).RootElement;
_api.Login(root.GetProperty("API_Key").GetString(), root.GetProperty("Secret_Key").GetString());

// var res = _api.Snapshots(new List<IContract>() {_api.Contracts.Futures["TXF"]["TXFR1"]}) //若誤用[0]就導它無法在array裡搜，硬用linq就報錯

Connecting as shioaji@sinopac on 203.66.91.161:80...


In [13]:
var GetChangePercentRankCodes = _api.Scanners(scannerType: ScannerType.ChangePercentRank, date: "2024-02-02", count: 20)
    .Where(x => x.close > 20 && x.total_amount > 500_000_000)
    .Select(x => new {code = x.code, name = x.name, close = x.close, total_amount =  Math.Round((decimal)x.total_amount / 100_000_000, 2)});

GetChangePercentRankCodes.Where(x => (decimal)x.GetType().GetProperty("total_amount").GetValue(x) >= 51)

index value 0 { code = 3324, name = 雙鴻, close = 425, total_amount = 51.28 } code 3324 name 雙鴻 close 425 total_amount 51.28 1 { code = 2421, name = 建準, close = 134.5, total_amount = 67.74 } code 2421 name 建準 close 134.5 total_amount 67.74

In [ ]:
using System.Net.Http;
try
{
    var client = new HttpClient();
    var response = client.GetAsync("https://openapi.twse.com.tw/v1/" + "exchangeReport/STOCK_DAY_AVG_ALL").Result;
    var json = response.Content.ReadAsStringAsync().Result;
    List<dynamic> OpenApiMonthlyAveragePrice = JsonConvert.DeserializeObject<List<dynamic>>(json);
    // OpenApiMonthlyAveragePrice.Where(x=> x.Code == "2330").Select( x => new {code = x.code, name = x.MonthlyAveragePrice})
    // OpenApiMonthlyAveragePrice.Any(x => x.Code == "2330")
    OpenApiMonthlyAveragePrice.Where(x => x.Code == "2330").Select( x => x );
    // OpenApiMonthlyAveragePrice.Take(3);
}
catch (Exception ex)
{
    Console.WriteLine($"Failed {ex.Message}");
}

OpenApiMonthlyAveragePrice.Where(x => x.Code == "2330").Select( x => x )

In [ ]:
using System.Threading;

List<Sinopac.Shioaji.FuturePosition> _src = _api.ListPositions(_api.FutureAccount);
var argStp = 10M;
// 若要mxf 雙sell + BP才監控的話就可用 src.Count(x=x.code.Contains() && .....).Any()
if(_src.Any(x => x.code.Contains("MXF")))
{
    //無關痛癢寫點東西
    Console.WriteLine($"test{_src.Where(x => x.code.Contains("MXF")).Select(x => x.pnl)}.");

    // 建容器存自執行以來的tick，檢查是否漏記錄，但若改用callback應該就會是準備不必特別記
    // List<double> _temp = new List<double>();

    //很蠢還在定時呼叫快照存close，用callback應該就會是準備不必特別記
    // double spotClose = (double)_api.Snapshots(new List<IContract>() { _api.Contracts.Futures["TXF"]["TXFR1"] })[0].close;
    // _temp.Add(spotClose);
    // _temp.Reverse();
    // string result = string.Join(", ", _temp.Take(10));
    // Console.WriteLine($"現在{spotClose}點 \t 過去 10 ticks {result}\n高點{_temp.Max()} since start");
    decimal argEntryPrice = _src.Where(x => x.code.Contains("MXF")).Select(x => x.price).SingleOrDefault();
    if (17950 >= argEntryPrice + argStp)
    {
        argEntryPrice += 2;
        Console.WriteLine($"已拉開故到{argEntryPrice}才會被掃出場");
        while (true)
        {
            spotClose = (double)_api.Snapshots(new List<IContract>() { _api.Contracts.Futures["TXF"]["TXFR1"] })[0].close;
            _temp.Add(spotClose);
            if (spotClose <= (_temp.Max() - argStp))
            {
                string desc = spotClose >= argEntryPrice ? "停利" : "保本";
                Console.WriteLine($"★{desc}出在{spotClose}。");
                break;
            }
            else if (spotClose <= argEntryPrice)
            {
                Console.WriteLine($"★保本出在{spotClose}。");
                break;
            }
            else
            {
                Console.WriteLine(spotClose);
                Thread.Sleep(5_000);
            }
        }
        break;
    }
    else if (spotClose <= argEntryPrice - argStp)
    {
        Console.WriteLine($"★停損，出在{spotClose}。");
        break;
    }
    else Thread.Sleep(5_000);
}

# 這區是OK的別誤刪

In [ ]:
using System.Threading;

public void MxfMock(double argEntryPrice, double argStp)
{
    List<double> _temp = new List<double>();
    while (true)
    {
        double spotClose = (double)_api.Snapshots(new List<IContract>() { _api.Contracts.Futures["TXF"]["TXFR1"] })[0].close;
        _temp.Add(spotClose);
        _temp.Reverse();
        string result = string.Join(", ", _temp.Take(10));
        Console.WriteLine($"現在{spotClose}點 \t 過去 10 ticks {result}\n高點{_temp.Max()} since start");
        if (spotClose >= argEntryPrice + argStp)
        {
            argEntryPrice += 2;
            Console.WriteLine($"已拉開故到{argEntryPrice}才會被掃出場");
            while (true)
            {
                spotClose = (double)_api.Snapshots(new List<IContract>() { _api.Contracts.Futures["TXF"]["TXFR1"] })[0].close;
                _temp.Add(spotClose);
                if (spotClose <= (_temp.Max() - argStp))
                {
                    string desc = spotClose >= argEntryPrice ? "停利" : "保本";
                    Console.WriteLine($"★{desc}出在{spotClose}。");
                    break;
                }
                else if (spotClose <= argEntryPrice)
                {
                    Console.WriteLine($"★保本出在{spotClose}。");
                    break;
                }
                else
                {
                    Console.WriteLine(spotClose);
                    Thread.Sleep(5_000);
                }
            }
            break;
        }
        else if (spotClose <= argEntryPrice - argStp)
        {
            Console.WriteLine($"★停損，出在{spotClose}。");
            break;
        }
        else Thread.Sleep(5_000);
    }
    Console.WriteLine("★★小台積期：" + _api.Snapshots(new List<IContract>() { _api.Contracts.Futures["QFF"]["QFFR1"] })[0].close);
}
MxfMock(17950.0, 10.0)